In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import geopandas
import pandas
import rasterio

from affine import Affine
from open_gira.io import write_raster_ds as write_ds, read_raster_ds as read_ds
from open_gira.geometry import (
    resample_sum,
    clip_array,
    repeat_2d,
    floor_int,
    zero_divide,
    rasterize,
)
from snail.damages import PiecewiseLinearDamageCurve

In [ ]:
base_dir = Path("../results/input")
# see scripts/building_damage.py for alternative implementation

# inputs
rr_val_150ss_tif = base_dir / "giri/THA/bem_5x5_valfis_res__THA.tif"
rr_vol_3ss_tif = base_dir / "ghsl/THA/ghs_built_v_res_3ss__THA.tif"
nr_val_150ss_tif = base_dir / "giri/THA/bem_5x5_valfis_nres__THA.tif"
nr_vol_3ss_tif = base_dir / "ghsl/THA/ghs_built_v_nres_3ss__THA.tif"
flood_1ss_tif = base_dir / "footprints/JBA/Raster/TH_FLRF_ChaoPhraya2011_RD_01.tif"

# outputs
rr_val_3ss_tif = base_dir / "giri/THA/bem_3ss_valfis_res__THA.tif"
nr_val_3ss_tif = base_dir / "giri/THA/bem_3ss_valfis_nres__THA.tif"
rr_dmg_val_1ss_tif = base_dir / "giri/THA/dmg_val_res_1ss.tif"
nr_dmg_val_1ss_tif = base_dir / "giri/THA/dmg_val_nres_1ss.tif"

In [ ]:
def process_val_3ss(val_150ss_tif, vol_3ss_tif, val_3ss_tif):
    val_150ss, val_150ss_ds = read_ds(val_150ss_tif, replace_nodata=True)
    vol_3ss, vol_3ss_ds = read_ds(vol_3ss_tif, replace_nodata=True)
    volume_3ss, t_3ss = calculate_volume_3ss(
        val_150ss, val_150ss_ds, vol_3ss, vol_3ss_ds
    )
    write_ds(val_3ss_tif, volume_3ss, t_3ss)


def calculate_volume_3ss(val_150ss, val_150ss_ds, vol_3ss, vol_3ss_ds):
    # lon, lat of volume_3ss top left
    vol_3ss_ul_xy = vol_3ss_ds.transform * (0, 0)
    # col, row in value_150ss_all, inset one extra
    val_150ss_ul_cr = floor_int(~val_150ss_ds.transform * (vol_3ss_ul_xy)) + 1
    # lon, lat of that val_150ss_all pixel - this is our new top left
    ul_xy_150ss = val_150ss_ds.transform * val_150ss_ul_cr
    # col, row in vol_3ss_all
    vol_3ss_ul_cr = floor_int(~vol_3ss_ds.transform * ul_xy_150ss)
    # lon, lat of that vol_3ss_all pixel - new top left for 3ss purposes (tiny bit offset)
    ul_xy_3ss = vol_3ss_ds.transform * vol_3ss_ul_cr
    ul_xy_150ss, ul_xy_3ss

    # Clip out vol array
    col_idx, row_idx = vol_3ss_ul_cr
    vol_3ss = vol_3ss[row_idx:, col_idx:]
    vol_3ss = clip_array(vol_3ss, 50)
    # Resample vol to coarse-scale, "sum"
    vol_150ss = resample_sum(vol_3ss, 50)

    a, b, _, d, e, _ = vol_3ss_ds.transform[:6]
    # t_150ss = Affine(a * 50, b, ul_xy_150ss[0], d, e * 50, ul_xy_150ss[1])
    t_3ss = Affine(a, b, ul_xy_3ss[0], d, e, ul_xy_3ss[1])

    col_idx, row_idx = val_150ss_ul_cr
    ncols, nrows = vol_150ss.shape
    val_150ss = val_150ss[col_idx : col_idx + ncols, row_idx : row_idx + nrows]

    if val_150ss.shape != vol_150ss.shape:
        print("CHKS", val_150ss.shape, vol_150ss.shape)
        assert False

    # Calculate val per unit vol
    # val_per_vol_150ss = val_150ss / vol_150ss
    val_per_vol_150ss = np.divide(
        val_150ss, vol_150ss, out=np.zeros_like(val_150ss), where=vol_150ss != 0
    )
    # Resample to fine-scale val per vol, "nearest"
    val_per_vol_3ss = repeat_2d(val_per_vol_150ss, 50)
    # Calculate fine-scale val
    val_3ss = val_per_vol_3ss * vol_3ss

    return val_3ss, t_3ss


# ~15s
process_val_3ss(rr_val_150ss_tif, rr_vol_3ss_tif, rr_val_3ss_tif)
process_val_3ss(nr_val_150ss_tif, nr_vol_3ss_tif, nr_val_3ss_tif)

## Flood intersection


In [ ]:
flood_1ss, flood_1ss_ds = read_ds(flood_1ss_tif, replace_nodata=True)

In [ ]:
nr_val_3ss, nr_val_3ss_ds = read_ds(nr_val_3ss_tif)
rr_val_3ss, rr_val_3ss_ds = read_ds(rr_val_3ss_tif)
t_3ss = rr_val_3ss_ds.transform

In [ ]:
rr_damage_curve = PiecewiseLinearDamageCurve.from_csv(
    "../config/damage_curves/flood/residential_asia.csv",
    intensity_col="inundation_depth_(m)",
    damage_col="damage_fraction",
)
nr_damage_curve = PiecewiseLinearDamageCurve.from_csv(
    "../config/damage_curves/flood/commercial_asia.csv",
    intensity_col="inundation_depth_(m)",
    damage_col="damage_fraction",
)

In [ ]:
def clip_rescale_1ss_3ss(t_1ss, t_3ss, data_1ss, data_3ss):
    # lon, lat of footprint top left
    t_1ss_ul_xy = t_1ss * (0, 0)
    # col, row in value_3ss
    t_3ss_ul_cr = floor_int(~t_3ss * (t_1ss_ul_xy))
    # lon, lat of that pixel - this is our new top left
    footprint_ul_xy_3ss = t_3ss * t_3ss_ul_cr
    # col, row in flood_1ss
    t_1ss_ul_cr = floor_int(~t_1ss * footprint_ul_xy_3ss)

    # lon, lat of that 1ss pixel - new top left for 1ss purposes (tiny bit offset)
    ul_xy_1ss = t_1ss * t_1ss_ul_cr

    # clip to match coarser array extent
    data_1ss_clipped = clip_array(data_1ss, 3)
    data_1ss_height, data_1ss_width = data_1ss_clipped.shape

    # lon, lat of footprint lower right
    t_1ss_lr_xy = t_1ss * (data_1ss_width, data_1ss_height)
    # col, row in value_3ss
    t_3ss_lr_cr = floor_int(~t_3ss * (t_1ss_lr_xy))

    ulc, ulr = t_3ss_ul_cr
    lrc, lrr = t_3ss_lr_cr

    data_3ss_clipped = data_3ss[ulr:lrr, ulc:lrc]
    data_3ss_as_1ss = repeat_2d(data_3ss_clipped, 3) / 9

    # Adapt transform to new top-left and resolution
    a, b, _, d, e, _ = t_1ss[:6]
    t_1ss_clipped = Affine(a, b, ul_xy_1ss[0], d, e, ul_xy_1ss[1])

    return data_1ss_clipped, data_3ss_as_1ss, t_1ss_clipped


def calculate_damage_val(flood_1ss, t_1ss, val_3ss, t_3ss, damage_curve):
    flood_1ss_clipped, val_1ss, t_1ss_clipped = clip_rescale_1ss_3ss(
        t_1ss, t_3ss, flood_1ss, val_3ss
    )

    if val_1ss.shape != flood_1ss_clipped.shape:
        print("CHKS", val_1ss.shape, flood_1ss_clipped.shape)
        assert False

    damage_fraction_1ss = damage_curve.damage_fraction(flood_1ss_clipped)
    damage_value_1ss = val_1ss * damage_fraction_1ss
    return damage_value_1ss, t_1ss_clipped


rr_dmg_val_1ss, t_1ss_clipped = calculate_damage_val(
    flood_1ss,
    flood_1ss_ds.transform,
    rr_val_3ss,
    rr_val_3ss_ds.transform,
    rr_damage_curve,
)
nr_dmg_val_1ss, t_1ss_clipped = calculate_damage_val(
    flood_1ss,
    flood_1ss_ds.transform,
    nr_val_3ss,
    nr_val_3ss_ds.transform,
    nr_damage_curve,
)

In [ ]:
write_ds(rr_dmg_val_1ss_tif, rr_dmg_val_1ss, t_1ss_clipped)
write_ds(nr_dmg_val_1ss_tif, nr_dmg_val_1ss, t_1ss_clipped)

In [ ]:
(
    rr_dmg_val_1ss.sum() / 1e9,
    nr_dmg_val_1ss.sum() / 1e9,
)

In [ ]:
rr_val_3ss.sum() / 1e9, nr_val_3ss.sum() / 1e9

# GVA downscaling

In [ ]:
"""
ADM1 damage values:

    exactextract \
        -p ../../admin-boundaries/tha_adm1.shp \
        -r dmg_val_1ss.tif \
        -f GID_1 \
        -s sum \
        -o dmg_val_1ss.csv

ADM1 total built volume:

    exactextract \
        -p ../../admin-boundaries/tha_adm1.shp \
        -r ../../ghsl/THA/GHS_BUILT_V_E2020_GLOBE_R2023A_4326_3ss_V1_0__THA.tif \
        -f GID_1 \
        -s sum \
        -o ghs_built_v_3ss.csv
"""

In [ ]:
adm1_vol = pandas.read_csv("input/giri/THA/ghs_built_v_3ss.csv").rename(
    columns={"sum": "built_volume"}
)

In [ ]:
adm1 = geopandas.read_file("input/admin-boundaries/tha_adm1.shp").merge(
    adm1_vol, on="GID_1"
)[["GID_1", "NAME_1", "built_volume", "geometry"]]

In [ ]:
with rasterio.open(
    "input/ghsl/THA/GHS_BUILT_V_E2020_GLOBE_R2023A_4326_3ss_V1_0__THA.tif"
) as vol_3ss_ds:
    vol_3ss = vol_3ss_ds.read(1)

In [ ]:
vol_adm1_3ss = rasterize(adm1, "built_volume", vol_3ss_ds)

In [ ]:
plt.imshow(vol_adm1_3ss)

In [ ]:
adm1_gva = pandas.read_csv(
    "/data/incoming/wenz-2023-dose-reported-subnational-output/DOSE_V2_THA.csv"
)
adm1_gva["ag_grp"] = adm1_gva["pop"] * adm1_gva.ag_grp_pc_usd
adm1_gva["man_grp"] = adm1_gva["pop"] * adm1_gva.man_grp_pc_usd
adm1_gva["serv_grp"] = adm1_gva["pop"] * adm1_gva.serv_grp_pc_usd

adm1_gva = geopandas.read_file("input/admin-boundaries/tha_adm1.shp").merge(
    adm1_gva, on="GID_1"
)[["GID_1", "NAME_1", "ag_grp", "man_grp", "serv_grp", "geometry"]]

In [ ]:
adm1_gva.drop(columns="geometry").to_csv("input/giri/THA/DOSE_V2_THA_rgva.csv")
adm1_gva.to_file("input/giri/THA/DOSE_V2_THA_rgva.gpkg", driver="GPKG")

In [ ]:
adm1_gva_ag_3ss = rasterize(adm1_gva, "ag_grp", vol_3ss_ds)
adm1_gva_man_3ss = rasterize(adm1_gva, "man_grp", vol_3ss_ds)
adm1_gva_serv_3ss = rasterize(adm1_gva, "serv_grp", vol_3ss_ds)

In [ ]:
gva_ag_3ss = zero_divide(vol_3ss, vol_adm1_3ss) * adm1_gva_ag_3ss
gva_man_3ss = zero_divide(vol_3ss, vol_adm1_3ss) * adm1_gva_man_3ss
gva_serv_3ss = zero_divide(vol_3ss, vol_adm1_3ss) * adm1_gva_serv_3ss

In [ ]:
write_ds("input/giri/THA/gva_ag_3ss.tif", gva_ag_3ss, vol_3ss_ds.transform)
write_ds("input/giri/THA/gva_man_3ss.tif", gva_man_3ss, vol_3ss_ds.transform)
write_ds("input/giri/THA/gva_serv_3ss.tif", gva_serv_3ss, vol_3ss_ds.transform)

In [ ]:
gva_ag_1ss = repeat_2d(gva_ag_3ss, 3) / 9
gva_man_1ss = repeat_2d(gva_man_3ss, 3) / 9
gva_serv_1ss = repeat_2d(gva_serv_3ss, 3) / 9

In [ ]:
# TODO figure out transform, check we're on the right grid, write out to files
# TODO compare with damage fraction, write out interruption
# TODO calculate per day, sum back to zonal stats
# TODO check totals (re-aggregate after disaggregation) maybe rescale???

In [ ]:
a, b, c, d, e, f = vol_3ss_ds.transform[:6]
gva_t_1ss = Affine(a / 3, b, c, d, e / 3, f)
gva_t_1ss

In [ ]:
write_ds("input/giri/THA/gva_ag_1ss.tif", gva_ag_1ss, gva_t_1ss)
write_ds("input/giri/THA/gva_man_1ss.tif", gva_man_1ss, gva_t_1ss)
write_ds("input/giri/THA/gva_serv_1ss.tif", gva_serv_1ss, gva_t_1ss)

In [ ]:
"""
gdalwarp -te 99.2393056 13.2781945 101.5259723 17.6765279 gva_man_1ss.tif gva_man_1ss_clipped.tif
gdal_calc.py -A nres_dmg_frac_1ss.tif -B gva_man_1ss_clipped.tif --outfile=disruption_man_1ss.tif --calc="(A>0.1)*B"


gdalwarp -te 99.2393056 13.2781945 101.5259723 17.6765279 gva_ag_1ss.tif gva_ag_1ss_clipped.tif
gdal_calc.py -A nres_dmg_frac_1ss.tif -B gva_ag_1ss_clipped.tif --outfile=disruption_ag_1ss.tif --calc="(A>0.1)*B"


gdalwarp -te 99.2393056 13.2781945 101.5259723 17.6765279 gva_serv_1ss.tif gva_serv_1ss_clipped.tif
gdal_calc.py -A nres_dmg_frac_1ss.tif -B gva_serv_1ss_clipped.tif --outfile=disruption_serv_1ss.tif --calc="(A>0.1)*B"


for sector in serv ag man
    exactextract \
        -p ../../admin-boundaries/tha_adm1.shp \
        -r disruption_{$sector}_1ss.tif \
        -f GID_1 \
        -s sum \
        -o disruption_{$sector}_1ss.csv
end
"""

In [ ]:
"""
gdalwarp -te 99.2393056 13.2781945 101.5259723 17.6765279  ../../footprints/JBA/Raster/TH_FLRF_ChaoPhraya2011_RD_01.tif ../../footprints/JBA/Raster/TH_FLRF_ChaoPhraya2011_RD_01_clipped.tif
for sector in serv ag man
    gdal_calc.py \
        -A ../../footprints/JBA/Raster/TH_FLRF_ChaoPhraya2011_RD_01_clipped.tif \
        -B gva_{$sector}_1ss_clipped.tif \
        --outfile=disruption_0.3m_{$sector}_1ss.tif \
        --calc="(A>0.3)*B"
    exactextract \
        -p ../../admin-boundaries/tha_adm1.shp \
        -r disruption_0.3m_{$sector}_1ss.tif \
        -f GID_1 \
        -s sum \
        -o disruption_0.3m_{$sector}_1ss.csv
end

for sector in serv ag man
    gdal_calc.py \
        -A nres_dmg_frac_1ss.tif \
        -B gva_{$sector}_1ss_clipped.tif \
        --outfile=disruption_dmg_{$sector}_1ss.tif \
        --calc="A*B"
    exactextract \
        -p ../../admin-boundaries/tha_adm1.shp \
        -r disruption_dmg_{$sector}_1ss.tif \
        -f GID_1 \
        -s sum \
        -o disruption_dmg_{$sector}_1ss.csv
end
"""